# Chapter 5: LangGraph for Complex Agent Workflows

## The Director's Chair for Your AI Crew

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPOSITORY/blob/main/notebooks/05-LangGraph-for-Agent-Workflows.ipynb)

You've built single agents and linear crews. But what happens when the process isn't a straight line? What if you need loops, branches, and more complex logic? Welcome to **LangGraph**, the library that gives you the power to direct your agents with precision.

## 📈 Beyond Linearity: Why We Need Graphs

Real-world workflows are rarely linear. Consider a content creation process:

1.  A writer creates a draft.
2.  An editor reviews it.
3.  **If** there are issues, it goes back to the writer for revision (a **loop**).
4.  **Otherwise**, it gets published (a **branch**).

LangGraph allows us to model these complex, cyclical, and conditional workflows, which are essential for building robust and intelligent agents.

## 🧱 The Building Blocks of LangGraph

LangGraph has three core components:

1.  **State:** A central Python object (often a `TypedDict`) that is passed between all the nodes in the graph. It's the shared memory of the workflow, and each node can update it.
2.  **Nodes:** The 'workers' of the graph. Each node is a Python function that receives the current state, performs an action, and returns a dictionary to update the state.
3.  **Edges:** The 'connectors' that define the flow. An edge connects one node to another. **Conditional edges** are special edges that route the flow to different nodes based on the current state.

In [ ]:
# Step 1: Install and Setup
!pip install langgraph langchain langchain_google_genai python-dotenv duckduckgo-search

import os
from dotenv import load_dotenv
if not load_dotenv():
    try:
        from google.colab import userdata
        os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
    except ImportError:
        print("Could not load API keys.")

## 🛠️ Hands-On: A Research Team with a Review Cycle

Let's build a research team that includes a review and revision loop. This is a classic use case for LangGraph.

**Workflow:**
1.  **Researcher:** Finds information on a topic.
2.  **Writer:** Writes a draft based on the research.
3.  **Reviewer:** Checks the draft. 
    - If it's good, the process ends.
    - If it needs work, it goes back to the writer with feedback.

In [ ]:
# Step 2: Define the State
from typing import TypedDict, List

class ResearchState(TypedDict):
    topic: str
    research_summary: str
    draft_article: str
    review_comments: str

In [ ]:
# Step 3: Define the Nodes
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun

llm = ChatGoogleGenerativeAI(model="gemini-pro")
search_tool = DuckDuckGoSearchRun()

def researcher_node(state: ResearchState):
    print("--- Node: Researcher ---")
    summary = search_tool.run(f"Summarize the latest news on {state['topic']}")
    return {"research_summary": summary}

def writer_node(state: ResearchState):
    print("--- Node: Writer ---")
    prompt = f"Based on this summary: {state['research_summary']}, write a short blog post."
    if state.get('review_comments'):
        prompt += f"\nIncorporate this feedback: {state['review_comments']}"
    
    draft = llm.invoke(prompt).content
    return {"draft_article": draft}

def reviewer_node(state: ResearchState):
    print("--- Node: Reviewer ---")
    prompt = f"Review this draft: {state['draft_article']}. Is it ready to publish? If not, provide feedback."
    review = llm.invoke(prompt).content
    if "ready" in review.lower():
        return {"review_comments": None}
    else:
        return {"review_comments": review}

In [ ]:
# Step 4: Define the Graph and Edges
from langgraph.graph import StateGraph, END

graph = StateGraph(ResearchState)

# Add nodes
graph.add_node("researcher", researcher_node)
graph.add_node("writer", writer_node)
graph.add_node("reviewer", reviewer_node)

# Define edges
graph.set_entry_point("researcher")
graph.add_edge("researcher", "writer")
graph.add_edge("writer", "reviewer")

# Define the conditional edge
def should_continue(state: ResearchState):
    if state.get('review_comments'):
        return "writer" # Loop back to the writer
    else:
        return END # End the process

graph.add_conditional_edges("reviewer", should_continue, {"writer": "writer", END: END})

# Compile the graph
runnable_graph = graph.compile()

In [ ]:
# Step 5: Run the Graph
inputs = {"topic": "The future of AI in medicine"}
final_state = runnable_graph.invoke(inputs)

print("\n--- Final Result ---")
print(final_state['draft_article'])

## 📊 Visualizing Your Workflow

One of the best features of LangGraph is that you can visualize your graph to understand and debug it. This helps ensure your logic is correct.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(runnable_graph.get_graph().draw_mermaid_png()))
except:
    # This can fail in some environments, so we'll just print the ASCII version
    print(runnable_graph.get_graph().print_ascii())

## ✅ LangGraph vs. CrewAI: When to Use Which

| Framework | Best For | Key Feature |
|---|---|---|
| **CrewAI** | Rapidly building role-based agent teams. | High-level, intuitive API for defining agents and tasks. |
| **LangGraph**| Building complex, custom workflows with loops and branches. | Fine-grained control over state and flow. |

They are not mutually exclusive! You can even use CrewAI agents as nodes within a LangGraph workflow for the best of both worlds.

In the next chapter, we'll dive into the vast ecosystem of **Hugging Face**, learning how to integrate pre-trained models and datasets into our agentic systems.